<a href="https://colab.research.google.com/drive/1ZYxj74KLyudSINCbrJnhaF0HhcvjIVxH?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center> <b> LABORATORIO 7 - Métodos Numéricos <br>
Grupo 32<br>
- 32. David Rativa <br>
- 37. Dylan Rivero <br>
- 50. Danny Yaluzan <br>

# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [1]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

In [2]:
A

array([[1.51915195e+01, 6.22108771e-01, 4.37727739e-01, 7.85358584e-01,
        7.79975808e-01, 2.72592605e-01, 2.76464255e-01, 8.01872178e-01,
        9.58139354e-01, 8.75932635e-01],
       [3.57817270e-01, 1.55009951e+01, 6.83462935e-01, 7.12702027e-01,
        3.70250755e-01, 5.61196186e-01, 5.03083165e-01, 1.37684496e-02,
        7.72826622e-01, 8.82641191e-01],
       [3.64885984e-01, 6.15396178e-01, 1.50753812e+01, 3.68824006e-01,
        9.33140102e-01, 6.51378143e-01, 3.97202578e-01, 7.88730143e-01,
        3.16836122e-01, 5.68098653e-01],
       [8.69127390e-01, 4.36173424e-01, 8.02147642e-01, 1.51437668e+01,
        7.04260971e-01, 7.04581308e-01, 2.18792106e-01, 9.24867629e-01,
        4.42140755e-01, 9.09315959e-01],
       [5.98092228e-02, 1.84287084e-01, 4.73552788e-02, 6.74880944e-01,
        1.55946248e+01, 5.33310163e-01, 4.33240627e-02, 5.61433080e-01,
        3.29668446e-01, 5.02966833e-01],
       [1.11894318e-01, 6.07193706e-01, 5.65944643e-01, 6.76406199e-03,
   

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [3]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B
c = invD @ b

In [4]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [5]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [6]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [7]:
# Check the result:

A @ x - b

array([ 1.11022302e-16,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -1.73472348e-17,  0.00000000e+00,
       -2.77555756e-17,  2.22044605e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction.
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [8]:
# ... ENTER YOUR CODE HERE ...
def Jacobi_iteration(A, b, eps = 1e-7, n_iter = 50):

    diag_1d = np.diag(A)
    B = -A.copy()
    np.fill_diagonal(B, 0)
    invD = np.diag(1./diag_1d)
    BB = invD @ B
    c = invD @ b

    x = np.ones(n)
    for _ in range(n_iter):
        x = BB @ x + c
    return x

Checkers

In [9]:
x = Jacobi_iteration(A, b)
print(x)

np.testing.assert_allclose(A@x, b)
np.testing.assert_allclose(x, xx)

[ 0.03919429  0.03780037  0.04283232  0.02365951  0.05745031 -0.00030244
 -0.00577279  0.03177549 -0.00422849  0.05284648]


In [10]:
for k in range(1, 16):
    A1 = A + np.diagflat([-k]*n)
    print(np.linalg.norm(np.diag(1./np.diag(A1))@(-A1.copy()+np.diag(np.diag(A1)))),
          np.linalg.norm(Jacobi_iteration(A1, b)-np.linalg.solve(A1, b)))


0.38959181027260875 2.075447300981933e-17
0.4185783948614869 1.887508484344425e-17
0.45222840254738184 3.559354985867845e-17
0.4917667095178099 3.108315165532373e-17
0.5388887887486234 3.987034638053695e-17
0.5960110344093967 1.015041909473326e-15
0.6667001660296402 2.77091368904554e-13
0.7564517359241751 1.499052948993222e-10
0.8742017351588475 2.0161596094029004e-07
1.0355299928250667 0.000919171740567776
1.2702850939751231 23.481633679539158
1.6439565658213244 8260242.7936336445
2.3348091117608556 261149802433164.75
4.080768845910033 1.3716691464130678e+26
30.715327603064885 1.739863582849129e+61


# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration.

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [11]:
# ... ENTER YOUR CODE HERE ...

def seidel_iteration(A, b, eps = 1e-7, n_iter = 50):
    x = np.ones(b.shape[0])

    for _ in range(n_iter):
        for k in range(b.shape[0]):
              x[k] = (b[k]-np.dot(A[k][:k], x[:k])- np.dot(A[k][k+1:], x[k+1:]))/A[k,k]
    return x

Checkers

In [12]:
x = seidel_iteration(A,b)
print(x)

np.testing.assert_allclose(A@x, b)
np.testing.assert_allclose(x, xx)

[ 0.03919429  0.03780037  0.04283232  0.02365951  0.05745031 -0.00030244
 -0.00577279  0.03177549 -0.00422849  0.05284648]


In [13]:
for k in range(1, 16):
    A1 = A + np.diagflat([-k]*n)
    print(np.linalg.norm(np.diag(1./np.diag(A1))@(-A1.copy()+np.diag(np.diag(A1)))),
          np.linalg.norm(seidel_iteration(A1, b)-np.linalg.solve(A1, b)))

0.38959181027260875 1.5603850322462734e-17
0.4185783948614869 1.750373810356865e-17
0.45222840254738184 3.3393426912553537e-17
0.4917667095178099 2.5317455630777294e-17
0.5388887887486234 3.4312900515456327e-17
0.5960110344093967 2.716656719610089e-17
0.6667001660296402 2.5078606370050884e-17
0.7564517359241751 2.6479404645468863e-17
0.8742017351588475 5.015721274010177e-17
1.0355299928250667 2.9027475788800784e-17
1.2702850939751231 9.020562075079397e-17
1.6439565658213244 7.510070528825546e-17
2.3348091117608556 2.1898779786837934e-16
4.080768845910033 4.106336264861246e-09
30.715327603064885 2.2243276170648422e+114


# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [17]:
# ... ENTER YOUR CODE HERE ...
def safe_divide(num, denom, default_value=0):
    """Safely divide two numbers, avoiding division by very small or zero denominator."""
    if np.abs(denom) < 1e-10:  # Threshold for considering denominator as zero
        return default_value
    else:
        return num / denom

def minimum_residual_scheme(A, b, eps=1e-7, n_iter=50):
    x = np.ones(b.shape[0])

    for _ in range(n_iter):
        r = A @ x - b
        k = safe_divide(r @ A @ r, np.linalg.norm(A @ r) ** 2)
        x = x - k * r

    return x

Checkers

In [19]:
for k in range(1, 16):
    A1 = A + np.diagflat([-k]*n)
    print(np.linalg.norm(np.diag(1./np.diag(A1))@(-A1.copy()+np.diag(np.diag(A1)))),
          np.linalg.norm(minimum_residual_scheme(A1, b)-np.linalg.solve(A1, b)))

    #Revisar el nan

0.38959181027260875 3.861449326502864e-08
0.4185783948614869 9.165096173694062e-09
0.45222840254738184 1.739092364079409e-08
0.4917667095178099 3.467346864304144e-08
0.5388887887486234 2.0179038597882537e-08
0.5960110344093967 5.062685131483457e-08
0.6667001660296402 1.4087534815792388e-07
0.7564517359241751 8.585181216329117e-08
0.8742017351588475 1.4828665497308786e-07
1.0355299928250667 1.7250987770533834e-07
1.2702850939751231 6.793871504298367e-07
1.6439565658213244 4.3832605299965654e-07
2.3348091117608556 1.3771640310923207e-06
4.080768845910033 0.0002171791368337287
30.715327603064885 2.081804235084607
